Creado por: Carol Andrea Moncayo

Nombre del proyecto: Liquidador titulos valores

Enlace de video de Youtube: https://www.youtube.com/watch?v=Ro29ilWTjUk

In [ ]:
 # Import libraries
import pandas as pd
from datetime import datetime, timedelta
import calendar

json_url = "https://www.datos.gov.co/resource/pare-7x5i.json?$query=SELECT%20resolucion%2C%20fecha_resolucion%2C%20vigencia_desde%2C%20vigencia_hasta%2C%20interes_bancario_corriente%2C%20modalidad%20WHERE%20%60modalidad%60%20IN%20('CONSUMO%20Y%20ORDINARIO')"

try:
    df_interes_mensual = pd.read_json(json_url)
    display(df_interes_mensual.head())
except Exception as e:
    print(f"An error occurred while reading the JSON data: {e}")
    print("Please double check the URL and your internet connection.")

#Create a daraset
#Let´s build the values lists
valor_inicial = float(input("Ingrese el valor inicial del capital: "))
fecha_inicio = datetime.strptime(input("Ingrese la fecha de inicio (dd/mm/yyyy): "), "%d/%m/%Y")
interes_moratorio_factor = 1.5  # Factor for moratory interest
fecha_vencimiento = datetime.strptime(input("Ingrese la fecha de vencimiento (dd/mm/yyyy): "), "%d/%m/%Y")
fecha_liquidacion = datetime.strptime(input(f"Ingrese la fecha de liquidación (dd/mm/yyyy) (máximo {df_interes_mensual['vigencia_hasta'].max()}): "), "%d/%m/%Y")


valor_abonos = []
fecha_abonos = []

n_abonos = int(input("¿Cuántos abonos quieres registrar? "))
abonos = []
for i in range(n_abonos):
    fecha_abono = datetime.strptime(input(f"Fecha del abono {i + 1} (dd/mm/yyyy): "), "%d/%m/%Y")
    monto = float(input(f"Valor del abono {i + 1}: "))
    abonos.append((fecha_abono, monto))

# Ordenar abonos por fecha
abonos.sort(key=lambda x: x[0])

# Recorrer día a día hasta el último abono
fecha_final = abonos[-1][0] if abonos else fecha_inicio

# Setup inicial
capital = valor_inicial
interes_plazo_total = 0

fecha_actual = fecha_inicio
registro = []

print()
print("Fecha de liquidación:", fecha_liquidacion.strftime("%d/%m/%Y"))
print()


interes_moratorio_acumulado = 0
interes_plazo_acumulado = 0 # Initialize interest_plazo_acumulado
mora_iniciada = False


if fecha_inicio > fecha_vencimiento:
  print("La fecha de inicio no puede ser mayor a la fecha de vencimiento.")
  exit()

abonos_formateados = [(fecha.strftime('%d/%m/%Y'), valor) for fecha, valor in abonos] # Use the actual abonos list
print("Lista de abonos:", abonos_formateados)


# Convert 'vigencia_desde' and 'vigencia_hasta' to datetime objects in df_interes_mensual
df_interes_mensual['vigencia_desde'] = pd.to_datetime(df_interes_mensual['vigencia_desde']).dt.date
df_interes_mensual['vigencia_hasta'] = pd.to_datetime(df_interes_mensual['vigencia_hasta']).dt.date
df_interes_mensual['interes_bancario_corriente'] = df_interes_mensual['interes_bancario_corriente'].str.replace('%', '').astype(float)

# Calculate the duration of each interest rate period in days
def calculate_duration_in_days(start_date, end_date):
    start = datetime(start_date.year, start_date.month, start_date.day)
    end = datetime(end_date.year, end_date.month, end_date.day)
    return (end - start).days + 1


df_interes_mensual['duration_days'] = df_interes_mensual.apply(lambda row: calculate_duration_in_days(row['vigencia_desde'], row['vigencia_hasta']), axis=1)

# Calculate daily interest rate from annual rate
df_interes_mensual['interes_plazo_diario_rate'] = ((1 + df_interes_mensual['interes_bancario_corriente'] / 100)**(1/365) - 1)


while fecha_actual <= fecha_liquidacion: # Iterate until the liquidation date

    # Aplicar abonos en este día
    for fecha_abono, valor_abono in abonos:
        if fecha_abono.date() == fecha_actual.date():
            capital -= valor_abono


    # Find the corresponding interest rate for the current date
    interes_row = df_interes_mensual[
        (df_interes_mensual['vigencia_desde'] <= fecha_actual.date()) &
        (df_interes_mensual['vigencia_hasta'] >= fecha_actual.date())
    ]

    interes_plazo_diario = 0
    if not interes_row.empty:
        interes_plazo_diario = interes_row['interes_plazo_diario_rate'].iloc[0]

    # Calculate daily moratory interest based on daily plazo interest
    interes_moratorio_diario = interes_plazo_diario * interes_moratorio_factor


    # Check if moratory interest should start
    if fecha_actual > fecha_vencimiento and not mora_iniciada:
        interes_moratorio_acumulado += interes_plazo_acumulado # Add accumulated plazo interest to moratory interest
        interes_plazo_acumulado = 0 # Reset plazo interest
        mora_iniciada = True

    # Acumula interés diario (plazo before vencimiento, mora after)
    if not mora_iniciada:
        interes_plazo_acumulado += capital * interes_plazo_diario
    else:
        interes_moratorio_acumulado += capital * interes_moratorio_diario


    # Guardar registro diario
    registro.append({
        "fecha": fecha_actual.strftime("%d/%m/%Y"),
        "capital": capital,
        "interes_mora": interes_moratorio_acumulado,
        "interes_plazo_acumulado": interes_plazo_acumulado, # Use the correct variable name
        "interes_plazo_diario_rate": interes_plazo_diario * 100, # Add the calculated daily interest rate
        "interes_moratorio_diario_rate": interes_moratorio_diario * 100 # Add the calculated moratory daily interest rate
    })

    fecha_actual += timedelta(days=1)

# Combine interest columns and filter out zero total interest
for dia in registro:
  dia['total_interes'] = dia['interes_mora'] + dia['interes_plazo_acumulado']
  # Determine which daily interest rate to display and label it
  if datetime.strptime(dia['fecha'], "%d/%m/%Y").date() <= fecha_vencimiento.date():
      dia['Daily_Interest_Display'] = f"{round(dia['interes_plazo_diario_rate'], 4)}% (Plazo)"
  else:
      dia['Daily_Interest_Display'] = f"{round(dia['interes_moratorio_diario_rate'], 4)}% (Mora)"

registro_filtrado = [dia for dia in registro if dia['total_interes'] > 0]

# 📊 Mostrar resultados finales
print("\n📘 Historial del crédito por día:")
print(f"{'Fecha':<12} {'Capital':<10} {'Interés Mensual':<20} {'Interés':<20} {'Interés Acumulado':<20}")
for dia in registro_filtrado:
    # Find the monthly interest rate for the current date
    current_date = datetime.strptime(dia['fecha'], "%d/%m/%Y").date()
    interes_mensual_row = df_interes_mensual[
        (df_interes_mensual['vigencia_desde'] <= current_date) &
        (df_interes_mensual['vigencia_hasta'] >= current_date)
    ]
    interes_mensual_display = "N/A"
    if not interes_mensual_row.empty:
        interes_mensual_display = f"{round(interes_mensual_row['interes_bancario_corriente'].iloc[0], 2)}%"

    print(f"{dia['fecha']:<12} {round(dia['capital'], 2):<10} {interes_mensual_display:<20} {dia['Daily_Interest_Display']:<20} {round(dia['total_interes'], 2):<20}")


print("\nResumen final:")
print(f"Capital final: ${round(capital, 2):,.2f}")
print(f"Total interés de plazo + interés moratorio acumulado: ${round(interes_moratorio_acumulado, 2):,.2f}")
print(f"Total intereses + capital: ${round(interes_moratorio_acumulado + capital, 2):,.2f}")

,resolucion,fecha_resolucion,vigencia_desde,vigencia_hasta,interes_bancario_corriente,modalidad
0,428,2007-03-30T00:00:00.000,2007-04-01T00:00:00.000,2007-06-30T00:00:00.000,16.75%,CONSUMO Y ORDINARIO
1,1086,2007-06-29T00:00:00.000,2007-07-01T00:00:00.000,2007-09-30T00:00:00.000,19.01%,CONSUMO Y ORDINARIO
2,1742,2007-09-28T00:00:00.000,2007-10-01T00:00:00.000,2007-12-31T00:00:00.000,21.26%,CONSUMO Y ORDINARIO
3,2366,2007-12-28T00:00:00.000,2008-01-01T00:00:00.000,2008-03-31T00:00:00.000,21.83%,CONSUMO Y ORDINARIO
4,474,2008-03-31T00:00:00.000,2008-04-01T00:00:00.000,2008-06-30T00:00:00.000,21.92%,CONSUMO Y ORDINARIO


Ingrese el valor inicial del capital: 5000000
Ingrese la fecha de inicio (dd/mm/yyyy): 01/01/2021
Ingrese la fecha de vencimiento (dd/mm/yyyy): 10/01/2022
Ingrese la fecha de liquidación (dd/mm/yyyy) (máximo 2025-07-31T00:00:00.000): 31/10/2024
¿Cuántos abonos quieres registrar? 0

Fecha de liquidación: 31/10/2024

Lista de abonos: []

📘 Historial del crédito por día:
Fecha        Capital    Interés Mensual      Interés              Interés Acumulado   
01/01/2021   5000000.0  17.32%               0.0438% (Plazo)      2188.63             
02/01/2021   5000000.0  17.32%               0.0438% (Plazo)      4377.26             
03/01/2021   5000000.0  17.32%               0.0438% (Plazo)      6565.89             
04/01/2021   5000000.0  17.32%               0.0438% (Plazo)      8754.52             
05/01/2021   5000000.0  17.32%               0.0438% (Plazo)      10943.15            
06/01/2021   5000000.0  17.32%               0.0438% (Plazo)      13131.78            
07/01/2021   5000000

# Nueva sección

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

In [ ]:
import gradio as gr
import pandas as pd
from datetime import datetime, timedelta

# Assuming df_interes_mensual is already loaded and processed from the first cell

def calculate_basic_interest(valor_inicial, fecha_inicio_str, fecha_vencimiento_str, fecha_liquidacion_str, abonos_str):
    """
    Calculates the interest and generates a daily record (basic version for Gradio demo).

    Args:
        valor_inicial (float): The initial capital value.
        fecha_inicio_str (str): The start date in dd/mm/yyyy format.
        fecha_vencimiento_str (str): The due date in dd/mm/yyyy format.
        fecha_liquidacion_str (str): The liquidation date in dd/mm/yyyy format.
        abonos_str (str): A string representing abonos in the format 'dd/mm/yyyy:value;dd/mm/yyyy:value;...'.

    Returns:
        tuple: A tuple containing a pandas DataFrame with the daily record (for display), a string with the final summary, and the full DataFrame for export.
    """
    try:
        fecha_inicio = datetime.strptime(fecha_inicio_str, "%d/%m/%Y")
        fecha_vencimiento = datetime.strptime(fecha_vencimiento_str, "%d/%m/%Y")
        fecha_liquidacion = datetime.strptime(fecha_liquidacion_str, "%d/%m/%Y")

        interes_moratorio_factor = 1.5

        # Parse abonos string
        abonos = []
        if abonos_str:
            for abono_item in abonos_str.split(';'):
                try:
                    fecha_abono_str, monto_str = abono_item.split(':')
                    fecha_abono = datetime.strptime(fecha_abono_str.strip(), "%d/%m/%Y")
                    monto = float(monto_str.strip())
                    abonos.append((fecha_abono, monto))
                except ValueError:
                    return None, f"Error parsing abonos: Invalid format in '{abono_item}'. Expected 'dd/mm/yyyy:value'.", None

        abonos.sort(key=lambda x: x[0])

        capital = valor_inicial
        interes_moratorio_acumulado = 0
        interes_plazo_acumulado = 0
        mora_iniciada = False
        registro = []
        fecha_actual = fecha_inicio

        # Ensure df_interes_mensual is accessible and has the necessary columns
        if 'df_interes_mensual' not in globals():
             return None, "Error: df_interes_mensual DataFrame not found. Please run the first cell to load the data.", None

        # Ensure date columns are datetime objects and interest is float
        df_interes_mensual_processed = df_interes_mensual.copy()
        df_interes_mensual_processed['vigencia_desde'] = pd.to_datetime(df_interes_mensual_processed['vigencia_desde']).dt.date
        df_interes_mensual_processed['vigencia_hasta'] = pd.to_datetime(df_interes_mensual_processed['vigencia_hasta']).dt.date
        # Ensure '%' is removed and convert to float - handle potential errors
        df_interes_mensual_processed['interes_bancario_corriente'] = df_interes_mensual_processed['interes_bancario_corriente'].astype(str).str.replace('%', '', regex=False)
        df_interes_mensual_processed['interes_bancario_corriente'] = pd.to_numeric(df_interes_mensual_processed['interes_bancario_corriente'], errors='coerce').fillna(0) # Handle non-numeric

        # Calculate daily interest rate from annual rate
        df_interes_mensual_processed['interes_plazo_diario_rate'] = ((1 + df_interes_mensual_processed['interes_bancario_corriente'] / 100)**(1/365) - 1)


        while fecha_actual <= fecha_liquidacion:
            # Aplicar abonos en este día
            for fecha_abono, valor_abono in abonos:
                if fecha_abono.date() == fecha_actual.date():
                    total_interes_acumulado = interes_moratorio_acumulado + interes_plazo_acumulado
                    if valor_abono >= total_interes_acumulado:
                        sobrante = valor_abono - total_interes_acumulado
                        interes_moratorio_acumulado = 0
                        interes_plazo_acumulado = 0
                        capital -= sobrante
                    else:
                        if interes_moratorio_acumulado >= valor_abono:
                            interes_moratorio_acumulado -= valor_abono
                        else:
                            sobrante_abono = valor_abono - interes_moratorio_acumulado
                            interes_moratorio_acumulado = 0
                            interes_plazo_acumulado -= sobrante_abono


            # Find the corresponding interest rate for the current date
            interes_row = df_interes_mensual_processed[
                (df_interes_mensual_processed['vigencia_desde'] <= fecha_actual.date()) &
                (df_interes_mensual_processed['vigencia_hasta'] >= fecha_actual.date())
            ]

            interes_plazo_diario = 0
            interes_mensual_actual = "N/A"
            if not interes_row.empty:
                interes_plazo_diario = interes_row['interes_plazo_diario_rate'].iloc[0]
                interes_mensual_actual = f"{round(interes_row['interes_bancario_corriente'].iloc[0], 2)}%"


            interes_moratorio_diario = interes_plazo_diario * interes_moratorio_factor

            if fecha_actual.date() > fecha_vencimiento.date() and not mora_iniciada:
                interes_moratorio_acumulado += interes_plazo_acumulado
                interes_plazo_acumulado = 0
                mora_iniciada = True

            # Acumula interés diario (plazo before vencimiento, mora after)
            if not mora_iniciada:
                interes_plazo_acumulado += capital * interes_plazo_diario
            else:
                interes_moratorio_acumulado += capital * interes_moratorio_diario

            # Debugging print statements
            # print(f"Date: {fecha_actual.strftime('%d/%m/%Y')}, Mora Initiated: {mora_iniciada}, Daily Plazo Rate: {interes_plazo_diario}, Daily Mora Rate: {interes_moratorio_diario}")


            registro.append({
                "Fecha": fecha_actual.strftime("%d/%m/%Y"),
                "Capital": capital,
                "Interés Mensual Bancario": interes_mensual_actual,
                "Interés Diario (%)": round(interes_plazo_diario * 100 if not mora_iniciada else interes_moratorio_diario * 100, 4),
                "Total Interés Acumulado": interes_moratorio_acumulado + interes_plazo_acumulado # Keep total accumulated interest
            })

            fecha_actual += timedelta(days=1)

        df_registro = pd.DataFrame(registro)

        # Filter out rows with zero total accumulated interest for display
        df_registro_filtrado_formatted = df_registro[df_registro['Total Interés Acumulado'] > 0].copy()

        # Format columns for display
        df_registro_filtrado_formatted['Capital'] = df_registro_filtrado_formatted['Capital'].apply(lambda x: f"${x:,.2f}")
        df_registro_filtrado_formatted['Total Interés Acumulado'] = df_registro_filtrado_formatted['Total Interés Acumulado'].apply(lambda x: f"${x:,.2f}")

        # Add the (Plazo) or (Mora) label to the "Interés Diario (%)" column for display
        df_registro_filtrado_formatted['Interés Diario (%)'] = df_registro_filtrado_formatted.apply(
            lambda row: f"{row['Interés Diario (%)']}% (Plazo)" if datetime.strptime(row['Fecha'], '%d/%m/%Y').date() <= fecha_vencimiento.date() else f"{row['Interés Diario (%)']}% (Mora)",
            axis=1
        )


        final_summary = f"""
        Resumen final:
        Capital final: ${round(capital, 2):,.2f}
        Total interés de plazo + interés moratorio acumulado: ${round(interes_moratorio_acumulado + interes_plazo_acumulado, 2):,.2f}
        Total intereses + capital: ${round(capital + interes_moratorio_acumulado + interes_plazo_acumulado, 2):,.2f}
        """

        # Return the formatted dataframe for display, the summary, and the raw dataframe for export
        return df_registro_filtrado_formatted[['Fecha', 'Capital', 'Interés Mensual Bancario', 'Interés Diario (%)', 'Total Interés Acumulado']], final_summary, df_registro

    except ValueError as e:
        return None, f"Error parsing date: {e}. Please ensure dates are in dd/mm/yyyy format.", None
    except Exception as e:
        return None, f"An unexpected error occurred: {e}", None

def export_to_excel(df_registro):
    """
    Exports the daily record DataFrame to an Excel file.

    Args:
        df_registro (pd.DataFrame): The DataFrame containing the daily record.

    Returns:
        str: The file path to the saved Excel file.
    """
    if df_registro is None or df_registro.empty:
        return None
    excel_file = "liquidacion_credito.xlsx"
    df_registro.to_excel(excel_file, index=False)
    return excel_file

def clear_inputs():
    """Clears the input fields and outputs."""
    return "", "", "", "", "", None, None, None # Return empty values for all output components

with gr.Blocks() as demo_basic:
    gr.Markdown("# Basic Interest Calculator")
    # Added the explanation markdown here
    gr.Markdown("""
    👋 ¡Hola a todos! Mi nombre es Carol Moncayo y hoy quiero compartir con ustedes el proyecto que desarrollé durante el bootcamp.

    🧠 Se trata de una interfaz para liquidar títulos valores como pagarés, letras de cambio y otros documentos similares, especialmente cuando el asunto se lleva al ámbito judicial.

    📈 Ahora vamos a calcular los intereses.

    Para ello, utilizamos una tabla que realiza los cálculos por días, en lugar de por meses. Los intereses mensuales fueron obtenidos desde un archivo JSON de datos.gov.co, que contiene los intereses bancarios corrientes y ordinarios. Estos valores se convirtieron a tasas diarias para lograr mayor precisión.

    💡 ¿Cómo se aplica el abono?

    Primero se descuenta de los intereses acumulados.

    Si el abono no cubre todos los intereses, el saldo pendiente se sigue acumulando al día siguiente.

    Si el abono supera los intereses, el excedente se resta del capital, generando un nuevo capital sobre el cual se liquidan nuevamente los intereses.

    📌 Además, tenemos la fecha de vencimiento: A partir del día siguiente, se empiezan a cobrar intereses de mora, los cuales se calculan multiplicando el interés de plazo por 1.5, según la normativa vigente.

    📋 En la parte superior de la interfaz, encontramos un resumen
    """)


    with gr.Row():
        valor_inicial_input = gr.Number(label="Valor Inicial del Capital")
        fecha_inicio_input = gr.Textbox(label="Fecha de Inicio (dd/mm/yyyy)", placeholder="e.g., 01/01/2023")
        fecha_vencimiento_input = gr.Textbox(label="Fecha de Vencimiento (dd/mm/yyyy)", placeholder="e.g., 01/01/2024")
        fecha_liquidacion_input = gr.Textbox(label="Fecha de Liquidación (dd/mm/yyyy)", placeholder="e.g., 31/12/2024")
        abonos_input = gr.Textbox(label="Abonos (dd/mm/yyyy:value;...)", placeholder="e.g., 15/03/2023:1000;20/06/2023:500")

    with gr.Row(): # Added a new row for buttons
        calculate_button = gr.Button("Calculate Interest")
        clear_button = gr.Button("Reset") # Added the clear button


    output_summary = gr.Textbox(label="Summary") # Moved summary output before the table
    output_dataframe = gr.DataFrame(label="Daily Interest Record")
    export_button = gr.Button("Export to Excel")
    output_file = gr.File(label="Download Excel File")


    # State variable to hold the full DataFrame for export
    full_dataframe_state = gr.State(None)


    calculate_button.click(
        calculate_basic_interest,
        inputs=[valor_inicial_input, fecha_inicio_input, fecha_vencimiento_input, fecha_liquidacion_input, abonos_input],
        outputs=[output_dataframe, output_summary, full_dataframe_state] # Also output to the state variable
    )

    # Added the clear button click event
    clear_button.click(
        clear_inputs,
        inputs=[],
        outputs=[
            valor_inicial_input,
            fecha_inicio_input,
            fecha_vencimiento_input,
            fecha_liquidacion_input,
            abonos_input,
            output_summary, # Clear the summary
            output_dataframe, # Clear the dataframe
            full_dataframe_state # Clear the state variable
        ]
    )


    export_button.click(
        export_to_excel,
        inputs=[full_dataframe_state], # Use the state variable as input
        outputs=[output_file]
    )


demo_basic.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4d3543bc3c60addf34.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
